<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 13


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Inventory в C#, который будет представлять информацию о 
наличии товаров на складе. На основе этого класса разработать 2-3 производных 
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из 
классов должны быть реализованы новые атрибуты и методы, а также 
переопределены некоторые методы базового класса для демонстрации 
полиморфизма.

#### Дополнительное задание
Добавьте к сущестующим классам конструктора классов с использованием гетторов и сетторов и реализуйте взаимодействие объектов между собой

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте простое, сложное и множественное наследование

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [7]:
using System;
using System.Collections.Generic;

public interface IInventoryActions
{
    void AddItem(int amount);
    void RemoveItem(int amount);
    int GetFreeSpace();
}

public delegate void InventoryNotification(string message);

public class Inventory : IInventoryActions
{
    public int WarehouseId { get; set; }
    public string WarehouseName { get; set; }
    public int StorageCapacity { get; set; }
    public int UsedCapacity { get; protected set; }
    public string Location { get; set; }
    public bool IsTemperatureControlled { get; set; }
    public int NumberOfEmployees { get; set; }
    public List<string> StoredItems { get; set; } = new List<string>();

    public event InventoryNotification OnInventoryChanged;

    protected void Notify(string message)
    {
        OnInventoryChanged?.Invoke(message);
    }

    public Inventory(int id, string name, int capacity)
    {
        WarehouseId = id;
        WarehouseName = name;
        StorageCapacity = capacity;
        UsedCapacity = 0;
        Location = "Не указано";
        IsTemperatureControlled = false;
        NumberOfEmployees = 0;
    }

    public virtual void GetStorageStatus()
    {
        Console.WriteLine($"Склад {WarehouseName}: занято {UsedCapacity}/{StorageCapacity}");
        Console.WriteLine("Товары на складе: " + (StoredItems.Count > 0 ? string.Join(", ", StoredItems) : "нет"));
    }

    void IInventoryActions.AddItem(int amount) => AddItems(amount);
    void IInventoryActions.RemoveItem(int amount) => RemoveItems(amount);
    int IInventoryActions.GetFreeSpace() => StorageCapacity - UsedCapacity;

    public void AddItems(int amount)
    {
        if (UsedCapacity + amount <= StorageCapacity)
        {
            UsedCapacity += amount;
            Notify($"Добавлено {amount} единиц на склад {WarehouseName}");
        }
        else
        {
            Notify($"Недостаточно места на складе {WarehouseName}");
        }
    }

    public void RemoveItems(int amount)
    {
        if (amount <= UsedCapacity)
        {
            UsedCapacity -= amount;
            Notify($"Удалено {amount} единиц со склада {WarehouseName}");
        }
        else
        {
            Notify($"На складе {WarehouseName} нет столько товаров для удаления");
        }
    }

    public void AddStoredItem(string item)
    {
        StoredItems.Add(item);
        Notify($"Добавлен товар: {item}");
    }

    public void RemoveStoredItem(string item)
    {
        if (StoredItems.Remove(item))
            Notify($"Удален товар: {item}");
        else
            Notify($"Товар {item} не найден");
    }
}

public class PersonalInventory : Inventory
{
    public string OwnerName { get; set; }
    public bool IsPrivate { get; set; }
    public string SecurityCode { get; set; }
    public string AlarmSystem { get; set; }
    public List<string> AccessLog { get; set; } = new List<string>();

    public PersonalInventory(int id, string name, int capacity, string owner)
        : base(id, name, capacity)
    {
        OwnerName = owner;
        IsPrivate = true;
        SecurityCode = "0000";
        AlarmSystem = "Стандартная";
    }

    public void ChangeSecurityCode(string newCode)
    {
        SecurityCode = newCode;
        Notify($"Код безопасности для {OwnerName} обновлен.");
    }

    public void SetAlarm(string alarmType)
    {
        AlarmSystem = alarmType;
        Notify($"Сигнализация для {OwnerName} установлена: {AlarmSystem}");
    }

    public void LogAccess(string user)
    {
        AccessLog.Add($"{user} посетил склад {DateTime.Now}");
        Notify($"Склад посещён пользователем {user}");
    }

    public override void GetStorageStatus()
    {
        base.GetStorageStatus();
        Console.WriteLine($"Владелец: {OwnerName}, Приватный: {IsPrivate}, Код: {SecurityCode}, Сигнализация: {AlarmSystem}");
        Console.WriteLine("Лог доступа: " + (AccessLog.Count > 0 ? string.Join("; ", AccessLog) : "нет"));
    }
}

public class InventoryService<T> where T : IInventoryActions
{
    private readonly T _inventory;

    public InventoryService(T inventory)
    {
        _inventory = inventory;
    }

    public void AddItems(int amount) => _inventory.AddItem(amount);
    public void RemoveItems(int amount) => _inventory.RemoveItem(amount);
    public void ShowFreeSpace() => Console.WriteLine($"Свободное место: {_inventory.GetFreeSpace()} единиц");
}

void RunDemo()
{
    PersonalInventory pInv = new PersonalInventory(1, "Личный склад", 50, "Игорь");
    pInv.OnInventoryChanged += msg => Console.WriteLine($"[Событие]: {msg}");

    pInv.SetAlarm("Продвинутая");
    pInv.AddStoredItem("Телевизор");
    pInv.AddStoredItem("Холодильник");
    pInv.LogAccess("Игорь");

    InventoryService<PersonalInventory> service = new InventoryService<PersonalInventory>(pInv);
    service.AddItems(15);
    service.ShowFreeSpace();
    service.RemoveItems(5);
    service.ShowFreeSpace();

    pInv.GetStorageStatus();
}

RunDemo();


[Событие]: Сигнализация для Игорь установлена: Продвинутая
[Событие]: Добавлен товар: Телевизор
[Событие]: Добавлен товар: Холодильник
[Событие]: Склад посещён пользователем Игорь
[Событие]: Добавлено 15 единиц на склад Личный склад
Свободное место: 35 единиц
[Событие]: Удалено 5 единиц со склада Личный склад
Свободное место: 40 единиц
Склад Личный склад: занято 10/50
Товары на складе: Телевизор, Холодильник
Владелец: Игорь, Приватный: True, Код: 0000, Сигнализация: Продвинутая
Лог доступа: Игорь посетил склад 21.11.2025 17:25:54
